# Transformer Optimization

## Imports, Inits, and Functions

In [1]:
%load_ext autoreload
%autoreload 2
%config IPCompleter.greedy=True

import pdb, pickle, sys, warnings, tqdm, time, torch, json, gzip
warnings.filterwarnings(action='ignore')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from rank_bm25 import BM25Okapi
from sklearn.feature_extraction import _stop_words
import string

from tqdm._tqdm_notebook import tqdm_notebook
tqdm_notebook.pandas()

from sentence_transformers import SentenceTransformer, util, CrossEncoder
device = 'cuda' if torch.cuda.is_available() else 'cpu'